In [1]:
import pandas as pd
import numpy as np
import pickle

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from ml_features.features import create_features
from ml_simulation.dataset_split import customer_split
from ml_simulation.util import HiddenPrints
from ml_features.features import create_features
from ml_training.train_xgb import train_xgb
    

import warnings
warnings.filterwarnings('ignore')

# Load data
df_quotes = pd.read_csv('cleaned_quote_data.csv')
df_quotes['dt_creation_devis'] = pd.to_datetime(df_quotes['dt_creation_devis'])

In [2]:
split_result = customer_split(df_quotes)
df_train = split_result['train']
df_sim = split_result['simulation']


SPLIT CUSTOMERS: TRAIN vs SIMULATION: TRAINING SIZE 0.95
Split: 22708 train, 1180 sim customers


In [3]:
TRAIN = True

In [4]:
if TRAIN:
    with HiddenPrints():
        X_train = create_features(df_train)
    
    y_train = X_train['converted']
    X_train = X_train.drop(['numero_compte', 'converted'], axis=1)
    feature_names = X_train.columns.tolist()
    
    result = train_xgb(X_train, y_train, "simulation_poc")
    model = result['model']
    
    print(f"Model trained: {len(feature_names)} features")
else:
    with open('simulation_poc.pkl', 'rb') as file:
        model_data = pickle.load(file)
        model = model_data['model']
        feature_names = model_data['features']

✓ Model saved: simulation_poc.pkl
✓ AUC: 0.729
✓ F1 Score: 0.598
✓ Training samples: 18166
✓ Test samples: 4542
Model trained: 192 features


In [5]:
# Get non-converted customers from simulation pool
sim_conv = df_sim.groupby('numero_compte')['fg_devis_accepte'].max()
non_conv_customers = sim_conv[sim_conv == 0].index.tolist()
print(f"Non-converted customers: {len(non_conv_customers)}")

Non-converted customers: 923


In [6]:
def safe_predict(customer_id, quotes_df, model, feature_names):
    """
    Safely predict probability even if features are missing.
    """
    with HiddenPrints():
        features_df = create_features(quotes_df)
    
    cust_features = features_df[features_df['numero_compte'] == customer_id]
    
    if len(cust_features) == 0:
        cust_features = pd.DataFrame({'numero_compte': [customer_id]})
    
    X_dict = {}
    for feat in feature_names:
        if feat in cust_features.columns:
            X_dict[feat] = cust_features[feat].iloc[0] if len(cust_features) > 0 else 0
        else:
            X_dict[feat] = 0
    
    X_cust = pd.DataFrame([X_dict])
    
    # try:
    prob = model.predict_proba(X_cust[feature_names])[:, 1][0]
    # except:
    #     prob = 0.5
    
    return prob

In [7]:
# SCENARIO 4: FAST SAMPLING - SINGLE QUOTE CUSTOMERS
print("\n=== SCENARIO 4: FAST SAMPLING ===")

import random
random.seed(474)

# Step 1: Get all single-quote customers in one pass (no predictions yet)
single_quote_raw = []
for cust in non_conv_customers[:300]:
    quotes = df_sim[df_sim['numero_compte'] == cust]
    if len(quotes) == 1:
        single_quote_raw.append({
            'customer_id': cust,
            'product': quotes['famille_equipement_produit'].iloc[0],
            'price': quotes['mt_apres_remise_ht_devis'].sum(),
            'quotes': quotes
        })

print(f"✅ Found {len(single_quote_raw)} customers with exactly 1 quote")


=== SCENARIO 4: FAST SAMPLING ===
✅ Found 222 customers with exactly 1 quote


In [8]:
# Step 2: Sample diverse products FIRST (without predictions)
product_groups = {}
for cust in single_quote_raw:
    prod = cust['product']
    if prod not in product_groups:
        product_groups[prod] = []
    product_groups[prod].append(cust)

selected_raw = []
products_used = set()

for prod, cust_list in product_groups.items():
    if len(selected_raw) >= 5:
        break
    if prod not in products_used and cust_list:
        selected_raw.append(random.choice(cust_list))
        products_used.add(prod)

if len(selected_raw) < 5:
    remaining = [c for c in single_quote_raw if c not in selected_raw]
    selected_raw.extend(random.sample(remaining, 5 - len(selected_raw)))

# Step 3: Compute predictions ONLY for selected 5 customers
selected = []
for cust in selected_raw:
    baseline = safe_predict(cust['customer_id'], cust['quotes'], model, feature_names)
    selected.append({
        'customer_id': cust['customer_id'],
        'product': cust['product'],
        'price': cust['price'],
        'baseline': baseline,
        'quote_count': 1
    })

In [9]:
import numpy as np
import pandas as pd
import random
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display, clear_output

# ─── Config ───
OPTIONS = {
    "Actuel":        {"fam": None, "emoji": "📊", "color": "#6baed6", "title": "Situation actuelle"},
    "Poêle":         {"fam": "Poêle", "emoji": "🔥", "color": "#2ca02c", "title": "→ Poêle"},
    "Climatisation": {"fam": "Climatisation", "emoji": "❄️", "color": "#ff7f0e", "title": "→ Climatisation"},
    "ECS":           {"fam": "ECS : Chauffe-eau ou adoucisseur", "emoji": "💧", "color": "#1f77b4", "title": "→ ECS"},
}

# Make sure we have consistent lists / dicts
selected_ids = [c['customer_id'] for c in selected]

print("\n🎯 SELECTED 5 SINGLE-QUOTE CUSTOMERS:")
for i, cust in enumerate(selected):
    print(f" {i+1}. {cust['customer_id']} - {cust['product']} (baseline: {cust['baseline']:.3f})")

# Create baseline dataframe and dict
print("\n=== CREATING BASELINE PREDICTIONS ===")
baseline_results = []
for cust_id in selected_ids:
    quotes = df_sim[df_sim['numero_compte'] == cust_id].copy()
    prob = safe_predict(cust_id, quotes, model, feature_names)
    baseline_results.append({
        'customer_id': cust_id,
        'baseline_prob': prob
    })

baseline_df = pd.DataFrame(baseline_results)
print(baseline_df)

baseline_dict = dict(zip(baseline_df['customer_id'], baseline_df['baseline_prob']))

# Convert baseline values to array (in the order of selected_ids)
baseline_array = np.array([baseline_dict[cid] for cid in selected_ids])

# Also keep region list in same order
regions = []
for cid in selected_ids:
    quotes = df_sim[df_sim['numero_compte'] == cid].copy()
    reg = quotes['nom_region'].iloc[0] if 'nom_region' in quotes.columns and len(quotes) > 0 else 'Unknown'
    regions.append(reg)

# ─── Compute function ───
def compute(scenario_dropout=0.0, family=None):
    model_scen = model  # ← replace with real model if dropout affects it

    n_list = []

    for cid in selected_ids:
        quotes = df_sim[df_sim['numero_compte'] == cid].copy()

        if family is None:
            n_list.append(baseline_dict[cid])
        else:
            mod = quotes.copy()
            if len(mod) > 0:
                nr = mod.iloc[-1:].copy()
                nr['famille_equipement_produit'] = family
                mod = pd.concat([mod, nr], ignore_index=True)
            n_val = safe_predict(cid, mod, model_scen, feature_names)
            n_list.append(n_val)

    n = np.array(n_list)
    return {
        'b': baseline_array,           # fixed reference array
        'n': n,
        'r': regions,
        'delta_avg': np.mean(n - baseline_array) if family else 0.0
    }

# ─── Figure factory ───
def make_fig(data, key):
    info = OPTIONS[key]
    is_current = key == "Actuel"

    fig = make_subplots(
        1, len(selected_ids),
        subplot_titles=[f"{str(cid)[:8]}<br><sub>{r}</sub>" for cid, r in zip(selected_ids, data['r'])],
        horizontal_spacing=0.14,
        shared_yaxes=True
    )

    c_base = '#6baed6'
    c_new  = info['color']
    c_down = '#d62728'

    for i in range(len(selected_ids)):
        b_val = data['b'][i]
        n_val = data['n'][i]
        delta = n_val - b_val

        fig.add_trace(
            go.Bar(x=['Actuel'], y=[b_val], marker_color=c_base,
                   text=f"{b_val:.3f}", textposition='auto',
                   hovertemplate=f"Actuel<br>{b_val:.3f}<extra></extra>"),
            row=1, col=i+1
        )

        fig.add_trace(
            go.Bar(x=[key], y=[n_val],
                   marker_color=c_new if is_current or delta >= 0 else c_down,
                   text=f"{n_val:.3f}" + ("" if is_current else f"<br>{delta:+.3f}"),
                   textposition='auto',
                   hovertemplate=f"{key}<br>{n_val:.3f}" + ("" if is_current else f" ({delta:+.3f})") + "<extra></extra>"),
            row=1, col=i+1
        )

    delta_txt = f"(Δ {data['delta_avg']:+.3f})" if not is_current else ""
    fig.update_layout(
        title_text=f"Heat Pump {info['emoji']} {info['title']} {delta_txt}",
        height=540,
        template="plotly_white",
        barmode='group',
        margin=dict(t=110, b=60, l=50, r=30),
        showlegend=False
    )
    fig.update_yaxes(title_text="Probabilité", range=[0, 0.9])
    return fig

# ─── Widgets ───
slider = widgets.FloatSlider(
    value=0.0, min=0.0, max=0.5, step=0.025,
    description='Dropout:', readout_format='.3f',
    continuous_update=False, layout={'width': '380px'}
)

dd = widgets.Dropdown(
    options=list(OPTIONS.keys()),
    value="Actuel",
    description='Scénario :',
    layout={'width': '380px'}
)

output = widgets.Output()

def update(change=None):
    with output:
        output.clear_output(wait=True)
        key = dd.value
        fam = OPTIONS[key]['fam']
        data = compute(scenario_dropout=slider.value, family=fam)
        fig = make_fig(data, key)
        display(fig)

slider.observe(update, names='value')
dd.observe(update, names='value')

# Show UI
ui = widgets.VBox([
    widgets.HBox([slider, dd]),
    output
])

update()  # initial plot
display(ui)


🎯 SELECTED 5 SINGLE-QUOTE CUSTOMERS:
 1. CL00067306 - Pompe à chaleur (baseline: 0.601)
 2. CL00063542 - Photovoltaïque (baseline: 0.279)
 3. CL00314484 - Chaudière (baseline: 0.477)
 4. CL00203740 - Poêle (baseline: 0.338)
 5. CL00258714 - Autres (baseline: 0.329)

=== CREATING BASELINE PREDICTIONS ===
  customer_id  baseline_prob
0  CL00067306       0.601482
1  CL00063542       0.278705
2  CL00314484       0.476918
3  CL00203740       0.338481
4  CL00258714       0.329173
